In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO(r"Training\runs\detect\train\weights\best.pt")
video_path = r"assets\toll_gate.mp4"
output_path = r"assets\vehicle_count_output.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Store the track history
track_history = defaultdict(lambda: [])

## Get video frame rate
# fps = cap.get(cv2.CAP_PROP_FPS)
wait_time = int(1000 / fps)  # Waktu tunggu dalam milidetik antara setiap frame

# Koordinat untuk menggambar garis
coordinates = {
    "points": [
        ((0, 125), (0, 135)),
        ((75, 165), (75, 175)),
        ((140, 180), (140, 190)),
        ((205, 193), (205, 203)),
        ((273, 208), (273, 218)),
        ((340, 222), (340, 232)),
        ((410, 238), (410, 248)),
        ((484, 255), (484, 265)),
        ((562, 273), (562, 283))
    ],
    "gates": [
        ((0, 125), (65, 150)),
        ((65, 150), (140, 180)),
        ((140, 180), (205, 193)),
        ((205, 193), (273, 208)),
        ((273, 208), (340, 222)),
        ((340, 222), (410, 238)),
        ((410, 238), (484, 255)),
        ((484, 255), (562, 273))
    ]
}

# Struktur data untuk menghitung kendaraan
vehicle_counts = [{"BUS": 0, "CAR": 0} for _ in range(8)]
counted_vehicles = set()

def point_in_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

# Loop through the video frames
frame_count = 0
while cap.isOpened():
    # Break the loop if 'q' is pressed
    if cv2.waitKey(wait_time) & 0xFF == ord("q"):
        break
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        classes = results[0].boxes.cls.cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks and count vehicles
        for box, track_id, cls in zip(boxes, track_ids, classes):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 100:
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=5)

            # Check if vehicle crossed any gate
            for i, (gateA, gateB) in enumerate(coordinates["gates"]):
                gate_poly = [gateA, gateB, (gateB[0], gateB[1]+10), (gateA[0], gateA[1]+10)]
                if point_in_polygon(x, y, gate_poly) and (track_id, i) not in counted_vehicles:
                    vehicle_class = "BUS" if cls == 0 else "CAR"
                    vehicle_counts[i][vehicle_class] += 1
                    counted_vehicles.add((track_id, i))

        # Draw lines on annotated frames
        for pointA, pointB in coordinates["points"]:
            cv2.line(annotated_frame, pointA, pointB, color=(0, 0, 255), thickness=4)

        # Draw gates and add names on them
        for i, (gateA, gateB) in enumerate(coordinates["gates"]):
            cv2.line(annotated_frame, gateA, gateB, color=(0, 255, 0), thickness=4)
    
            # Defines the text position for the gate name.
            text_position = ((gateA[0] + gateB[0]) // 2, (gateA[1] + gateB[1]) // 2 - 10)
    
            # Add gate name text above the line
            gate_name = f"Gate {i+1}"
            cv2.putText(annotated_frame, gate_name, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # Display vehicle counts
        # Display vehicle counts for gates 1-4 on the left side
        for i in range(4):
            text = f"Gate {i+1}: BUS {vehicle_counts[i]['BUS']}, CAR {vehicle_counts[i]['CAR']}"
            cv2.putText(annotated_frame, text, (10, 30 + i*20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (191, 0, 255), 2)

        # Display vehicle counts for gates 5-8 on the right side
        for i in range(4, 8):
            text = f"Gate {i+1}: BUS {vehicle_counts[i]['BUS']}, CAR {vehicle_counts[i]['CAR']}"
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_x = annotated_frame.shape[1] - text_size[0] - 10
            cv2.putText(annotated_frame, text, (text_x, 30 + (i-4)*20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (191, 0, 255), 2)

        # Add text at the bottom center
        total_bus = sum(count["BUS"] for count in vehicle_counts)
        total_car = sum(count["CAR"] for count in vehicle_counts)
        total_vehicles = total_bus + total_car

        # Menambahkan tulisan di bagian bawah tengah
        text_bottom = f"BUS = {total_bus} || CAR = {total_car} || TOTAL = {total_vehicles}"
        text_size = cv2.getTextSize(text_bottom, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
        text_x = (annotated_frame.shape[1] - text_size[0]) // 2
        text_y = annotated_frame.shape[0] - 10

        # Draw text on annotated frames
        cv2.putText(annotated_frame, text_bottom, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # # Display progress
        # frame_count += 1
        # progress = f"Progress: {frame_count}/{total_frames} frames"
        # cv2.putText(annotated_frame, progress, (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Write the frame to the output video
        out.write(annotated_frame)

    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()

print()
print(f"Video processing completed. Output saved to: {output_path}")

# Print result after video complete
print()
print("Vehicle Detection Results per Gate:")
for i in range(8):
    gate_result = f"Gate {i+1}: BUS {vehicle_counts[i]['BUS']}, CAR {vehicle_counts[i]['CAR']}"
    print(gate_result)

# Print total of transportation
total_bus = sum(count["BUS"] for count in vehicle_counts)
total_car = sum(count["CAR"] for count in vehicle_counts)
total_vehicles = total_bus + total_car
print(f"\nTotal:")
print(f"BUS = {total_bus}, CAR = {total_car}, TOTAL = {total_vehicles}")


0: 384x640 1 BUS, 1 CAR, 131.7ms
Speed: 9.8ms preprocess, 131.7ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 92.4ms
Speed: 3.3ms preprocess, 92.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 83.6ms
Speed: 0.0ms preprocess, 83.6ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 69.1ms
Speed: 2.7ms preprocess, 69.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 75.0ms
Speed: 0.0ms preprocess, 75.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 69.8ms
Speed: 3.0ms preprocess, 69.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 82.9ms
Speed: 2.7ms preprocess, 82.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BUS, 1 CAR, 82.3ms
Speed: 2.8ms preprocess, 82.3ms inference, 2.6ms postpr